In [1]:
## QICK LIBRARY
from qick import *

from pynq.buffer import allocate

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger, LFSR


import matplotlib.pyplot as plt
import numpy as np
import time

import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+
    
##### Load FGPA BitStream
soc = QickSoc('./qick_4x2_spin.bit')

#soc.config_clocks(1)
#print(soc)
#print(soc.qtt)

In [2]:
def qick_program (freq):
    reload(qasm)
    freq_int      = soc.freq2reg(freq)
    #period_cycles = soc.us2cycles(100/freq)
    period_cycles = soc.us2cycles(1000/freq)
    if (period_cycles > 65535):
        print('Waveform Longer than 16 bits > ', period_cycles)
    asm = f"""
    // set up pulse parameters
    //REG_WR w_freq imm #21845333
    REG_WR w_freq imm #{freq_int}
    //REG_WR w_phase imm #2000000000
      REG_WR w_phase imm #1500000000
    REG_WR w_gain imm #40000
    //REG_WR w_length imm #2000
    REG_WR w_length imm #{period_cycles}

    //REG_WR w_conf   imm  #1  // NoPHRST - NoPeriodic - DDS
    REG_WR w_conf   imm  #5  // NoPHRST - Periodic   - DDS
    //REG_WR w_conf   imm  #17 // PHRST   - NoPeriodic - DDS
    //REG_WR w_conf   imm  #21 // PHRST   - Periodic   - DDS

    REG_WR out_usr_time imm #500

    // play RF pulse
    WPORT_WR p0 r_wave
    WPORT_WR p1 r_wave

    REG_WR out_usr_time op -op(out_usr_time + #57)

    TRIG set p0
    TRIG set p1
    TRIG set p2

    REG_WR out_usr_time op -op(out_usr_time + w_length)

    TRIG clr p0
    TRIG clr p1
    TRIG clr p2

    TRIG set p3
    REG_WR out_usr_time op -op(out_usr_time + #10)
    TRIG clr p3


    JUMP HERE
    """
    p_txt, p_bin             = Assembler.str_asm2bin(asm)
    soc.tproc.Load_PMEM(p_bin)




In [35]:
freq = 24.5
#freq = 31.579
#freq = 12.485
#freq = 1.123

threshold      = 8000
period         = 1000 / freq
dead_time      = (period/2)
adc_freq       = 2 * 245.76
adc_period     = 1000 / adc_freq
adc_sr_period  = adc_period / 8
inhibit_clocks = int(dead_time / adc_period) + 1
inhibit_clocks = max(5, min(255, inhibit_clocks ) )

#print('inhibit_clocks',inhibit_clocks)

# Program qick Processor
qick_program(freq)

# Set QTT Threshold
soc.qtt.set_threshold(threshold)
# Set QTT Dead Time
soc.qtt.set_dead_time(inhibit_clocks)

##########################################################################
cfg_filter = 0
cfg_slope  = 0
cfg_inter  = 4
cfg_smp    = 4
cfg_invert = 0
soc.qtt.set_config(cfg_filter, cfg_slope, cfg_inter, cfg_smp, cfg_invert)

soc.qtt.arm()

soc.tproc.start()
print('period ', period, 'ns')
for ind in range(100):
    print(ind, end=" - ")
    tag_qty = soc.qtt.tag0_qty
    tags = soc.qtt.read_mem('TAG0')
    print(tag_qty)

soc.tproc.stop()
soc.qtt.disarm()


#smp = soc.qtt.read_mem('SMP').astype(int)
#soc.qtt.get_config()
#soc.qtt.print_axi_regs()




period  40.816326530612244 ns
0 - 313461
1 - 315576
2 - 319030
3 - 321628
4 - 322920
5 - 326190
6 - 327595
7 - 330348
8 - 331401
9 - 333460
10 - 334679
11 - 336588
12 - 337654
13 - 339743
14 - 344374
15 - 400564
16 - 404570
17 - 409283
18 - 411389
19 - 413981
20 - 415589
21 - 416941
22 - 418350
23 - 420898
24 - 421787
25 - 424426
26 - 426845
27 - 428938
28 - 467600
29 - 468394
30 - 472992
31 - 475448
32 - 477221
33 - 479516
34 - 480222
35 - 480897
36 - 482173
37 - 483203
38 - 484154
39 - 505493
40 - 504155
41 - 506066
42 - 506140
43 - 506718
44 - 507543
45 - 510147
46 - 510658
47 - 510725
48 - 511026
49 - 524161
50 - 516476
51 - 519468
52 - 519148
53 - 517963
54 - 518807
55 - 520703
56 - 520624
57 - 518785
58 - 517974
59 - 524287
60 - 516609
61 - 520692
62 - 519220
63 - 518260
64 - 518559
65 - 524287
66 - 524287
67 - 518214
68 - 519160
69 - 524287
70 - 516971
71 - 517885
72 - 518318
73 - 519663
74 - 519401
75 - 518941
76 - 518213
77 - 518355
78 - 518523
79 - 524287
80 - 516140
81 - 519

In [22]:
tag_qty = soc.qtt.tag0_qty
tags = soc.qtt.read_mem('TAG0')
print(tag_qty)
print(tags)


No Data to read in  TAG0
0
[]


In [ ]:
print()
